In [1]:
#imports
import pandas as pd
from aitextgen import aitextgen
import ipywidgets as widgets

In [2]:
# pandas options for bigger grids
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
# Download and load the model. 
# Set to_gpu=False if running on a CPU.
ai = aitextgen(model="minimaxir/magic-the-gathering", to_gpu=False)


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/vocab.json from cache at None
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/merges.txt from cache at None
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/tokenizer.json from cache at aitextgen/b1431d1de2fe5eabc1480965cc002593e0a1e07de5baaabbb66474fa2b5770d8.862a5b76401704bc81e91e440c522f95164df652d01d290de56ebc2896c47a98
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/tokenizer_config.json from cache at None


In [4]:
def generate_card_details(_):
    prompt=promptInput.value
    craziness = crazinessSlider.value
    numOfCards = batch_size = numOfCardsInput.value

    cards = ai.generate(n=numOfCards,
                        batch_size=batch_size,
                        schema=True,
                        prompt=prompt,
                        temperature=craziness,
                        return_as_list=True)

    cards = pd.DataFrame(list(cards))
    cards['text'] = cards.apply(lambda row: row['text'].replace("~", row['name']), axis=1)
    cards = cards[['name', 'manaCost', 'type', 'power', 'toughness','loyalty','text']]
    cardTable.value = cards.to_html()

In [5]:
# widgets
from ipywidgets import HBox, VBox, Label, Layout

wider = Layout(width="70vw")
less_wide = Layout(width="5vw")
promptInput = widgets.Text(placeholder='ex: <|type|>Creature - Goblin<|manaCost|>{2}{R}', layout=wider)
crazinessSlider = widgets.FloatSlider(.7,min=0.5,max=1.3)
numOfCardsInput = widgets.BoundedIntText(3,min=1,max=100,layout=less_wide)
generateCardsButton = widgets.Button(description="Generate")
cardTable = widgets.HTML()

generateCardsButton.on_click(generate_card_details)

display(HBox([Label("custom card prompt:"), promptInput]))
display(VBox([
    Label("(prompt list: <|name|>, <|manaCost|>, <|type|>, <|text|>,"),
    Label("<|power|>, <|toughness|>, <|loyalty|>)")
]))
display(HBox([Label("craziness:"), crazinessSlider]))
display(HBox([Label("number of cards to generate:"), numOfCardsInput]))
display(generateCardsButton)
display(cardTable)

Button(description='Generate', style=ButtonStyle())

HTML(value='')

In [9]:
saveCardsBtn = widgets.Button(description="save cards")
fileLocationInput = widgets.Text(value="My Cards.csv")

def saveCards(b):
    cards = pd.read_html(cardTable.value)[0]
    cards.to_csv(f'./output/{fileLocationInput.value}')

saveCardsBtn.on_click(saveCards)
display(HBox([fileLocationInput, saveCardsBtn]))